In [1]:
import torch
import numpy
import torch.nn.functional as F
from torch.nn import Linear 
import torch_geometric
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
import networkx as nx
from transformers import AutoModel, AutoTokenizer
import os
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup

/media/alvaro-montero/9408123808121A36/Ubuntu_Programs/Anaconda/envs/gnn_proj/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

def load_books_and_split_to_paragraphs(folder_path):
    # Lista para almacenar los párrafos de todos los libros
    all_paragraphs = []
    
    # Recorrer todos los archivos de la carpeta
    for filename in os.listdir(folder_path):
        # Verificar si el archivo tiene una extensión de texto válida
        if filename.endswith(".txt"):  # Puedes ajustar las extensiones si necesitas
            file_path = os.path.join(folder_path, filename)
            with open(file_path, 'r', encoding='utf-8') as file:
                content = file.read()
                # Dividir el contenido en párrafos usando doble salto de línea
                paragraphs = content.split("\n\n")
                # Limpiar espacios y agregar a la lista general
                all_paragraphs.extend([p.strip() for p in paragraphs if p.strip()])
    
    return all_paragraphs


In [3]:
def eliminar_etiquetas(text):
    pattern = re.compile(r'<[^>]+>|\([^)]+\)|--[^\-]+--')
    doc = pattern.sub('', text)
    return doc

# Removemos apostrofes para quedarnos con palabras como cant couldnt o dont
def remover_apostrofes(text):
    pattern = r"\\?'"
    doc = re.sub(pattern, '', text)
    return doc
# Nos quedamos con solo alfabeto eliminando caracteres especiales    
def remover_especiales(text):
    pattern = r"[^a-z.,\s]"
    doc = re.sub(pattern, ' ', text)
    return doc
# Quitamos dobles espacios para tener texto espaciado solamente por un espacio por palabra
def dobles_espacios(text):
    pattern = r'\s+'
    doc = re.sub(pattern, ' ', text)
    return doc

## Limpieza de datos

In [16]:
def clean_text_strip(text):
   
    #Método usando strip() y split()
  
    lines = text.strip().split('\r\n')
    return ' '.join(line.strip() for line in lines)

def clean_text_regex(text):

    # Reemplaza cualquier combinación de \r y \n con un espacio
    text = re.sub(r'[\r\n]+', ' ', text)
    # Elimina espacios múltiples
    text = re.sub(r'\s+', ' ', text)
    return text.strip()


#texts=load_books_and_split_to_paragraphs("/home/alvaro-montero/Escritorio/Repositorios/GNN_Project/Test text")

libros_ids = [1998, 2680, 1232, 5827, 8438, 66048]  # IDs de los libros seleccionados 3623, 2873

capitulos = []  # Lista para almacenar capítulos de todos los libros
ids_sin_capitulos = []  # IDs de libros sin capítulos

lista_parrafos = []

# Recorrer cada ID de libro
for libro_id in libros_ids:
    url = f'https://www.gutenberg.org/cache/epub/{libro_id}/pg{libro_id}-images.html'

    # Realizar la solicitud HTTP
    response = requests.get(url)

    if response.status_code == 200:
        # Crear un objeto BeautifulSoup para analizar el HTML
        soup = BeautifulSoup(response.content, 'html.parser')

        # Encontrar todos los divs con la clase "chapter"
        chapter_divs = soup.find_all('div', class_='chapter')

        if chapter_divs:
            # Lista para almacenar todos los capítulos de este libro
            libro_capitulos = []

            # Recorrer cada capítulo
            for chapter in chapter_divs:
                # Encontrar todos los párrafos dentro del capítulo
                # parrafos = [p.get_text() for p in chapter.find_all('p')]
                parrafos = [clean_text_regex(p.get_text()) for p in chapter.find_all('p')]

                # Añadir los párrafos del capítulo a la lista de capítulos
                if parrafos:  # Solo agregar capítulos con contenido
                    libro_capitulos.append(parrafos)

                    # Guardar cada parrafo en la lista general de todos los parrafos
                    for parrafo in parrafos:
                        longitud_parrafo = len(parrafo.split(" "))
                        if longitud_parrafo > 50: # Umbral para la cantidad de palabras por parrafo 
                            lista_parrafos.append(parrafo)
                            
                    print("len_lista_parrafos: ", len(lista_parrafos))
                    

            # Si se encontraron capítulos, almacenarlos con el ID del libro
            if libro_capitulos:
                capitulos.append((libro_capitulos))
        else:
            # Si no se encontraron capítulos, agregar el ID a la lista de libros sin capítulos
            ids_sin_capitulos.append(libro_id)
    else:
        print(f"Error al acceder al libro con ID {libro_id}. Código de estado: {response.status_code}")
"""
texts=load_books_and_split_to_paragraphs("/home/alvaro-montero/Escritorio/Repositorios/GNN_Project/Test text")
for text in texts:
    text=eliminar_etiquetas(text)
    text=remover_apostrofes(text)
    text=remover_especiales(text)
    text=dobles_espacios(text)
"""

#texts=[ parrafo for ]
texts=lista_parrafos

mat_len=len(texts)
#adjw_matrix=torch.zeros(mat_len,mat_len)
print(mat_len)

len_lista_parrafos:  24
len_lista_parrafos:  40
len_lista_parrafos:  54
len_lista_parrafos:  70
len_lista_parrafos:  104
len_lista_parrafos:  131
len_lista_parrafos:  171
len_lista_parrafos:  209
len_lista_parrafos:  255
len_lista_parrafos:  280
len_lista_parrafos:  310
len_lista_parrafos:  331
len_lista_parrafos:  352
len_lista_parrafos:  387
len_lista_parrafos:  393
len_lista_parrafos:  395
len_lista_parrafos:  395
len_lista_parrafos:  396
len_lista_parrafos:  398
len_lista_parrafos:  406
len_lista_parrafos:  413
len_lista_parrafos:  421
len_lista_parrafos:  426
len_lista_parrafos:  426
len_lista_parrafos:  428
len_lista_parrafos:  446
len_lista_parrafos:  453
len_lista_parrafos:  456
len_lista_parrafos:  464
len_lista_parrafos:  477
len_lista_parrafos:  485
len_lista_parrafos:  492
len_lista_parrafos:  496
len_lista_parrafos:  501
len_lista_parrafos:  512
len_lista_parrafos:  522
len_lista_parrafos:  526
len_lista_parrafos:  528
len_lista_parrafos:  533
len_lista_parrafos:  538
len_

In [17]:
texts[1000]

'But the Unjust man does not always choose actually the greater part, but even sometimes the less; as in the case of things which are simply evil: still, since the less evil is thought to be in a manner a good and the grasping is after good, therefore even in this case he is thought to be a grasping man, i.e. one who strives for more good than fairly falls to his share: of course he is also an unequal man, this being an inclusive and common term.'

## Procesamiento de datos

In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
bert_model = AutoModel.from_pretrained("bert-base-uncased")
def get_text_embeddings(texts):
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():  
        outputs = bert_model(**inputs)
        embeddings = outputs.last_hidden_state[:, 0, :]  # Usar el token [CLS]
    return embeddings

In [20]:
# Dividir textos en lotes
batch_size=1500
dataloader = torch.utils.data.DataLoader(texts, batch_size=batch_size, shuffle=False)
    
embeddings_list = []
with torch.no_grad():
        for batch in dataloader:
            inputs = tokenizer(batch, padding=True, truncation=True, return_tensors="pt")
            outputs = bert_model(**inputs)
            embeddings = outputs.last_hidden_state[:, 0, :]  # Usar el token [CLS]
            embeddings_list.append(embeddings)

    # Concatenar todos los embeddings en un solo tensor
node_embeddings = torch.cat(embeddings_list, dim=0)

print("Embeddings de nodos:", node_embeddings.shape)  # Debería ser (num_nodos, 768)


Embeddings de nodos: torch.Size([1559, 768])


In [21]:
"""
for i in range(len(node_embeddings)):
    for j in range(len(node_embeddings)):
        if i==j:
            continue
        else:
            adjw_matrix[i][j] = F.cosine_similarity(node_embeddings[i].unsqueeze(0),node_embeddings[j].unsqueeze(0))
"""

scores=torch.matmul(node_embeddings,node_embeddings.T)    

mask = torch.eye(scores.size(0), device=scores.device).bool()  # Matriz identidad
scores.masked_fill_(mask, float('-inf'))  # Establece -inf en la diagonal
adjw_matrix=F.softmax(scores,dim=1)

#adjw_matrix=torch.triu(adjw_matrix)
print(adjw_matrix)
print(adjw_matrix.shape)
  


tensor([[0.0000e+00, 1.5707e-06, 7.8099e-06,  ..., 5.2077e-21, 2.5639e-30,
         1.8543e-21],
        [4.2071e-07, 0.0000e+00, 1.7374e-08,  ..., 1.9928e-23, 3.2122e-37,
         1.6085e-22],
        [1.8212e-06, 1.5126e-08, 0.0000e+00,  ..., 3.4112e-21, 2.3567e-33,
         4.3093e-22],
        ...,
        [2.6521e-11, 3.7887e-13, 7.4494e-11,  ..., 0.0000e+00, 1.4643e-19,
         3.4151e-08],
        [3.7227e-13, 1.7413e-19, 1.4674e-15,  ..., 4.1750e-12, 0.0000e+00,
         2.7601e-12],
        [1.2070e-14, 3.9089e-15, 1.2028e-14,  ..., 4.3648e-11, 1.2373e-22,
         0.0000e+00]])
torch.Size([1559, 1559])


In [25]:
for i in range(mat_len):
    for j in range(mat_len):
        if adjw_matrix[i][j]<(1/mat_len):
            adjw_matrix[i][j]=0
print(adjw_matrix)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])


In [26]:
#adjw_matrix.cpu()
edge_index, edge_weight = torch_geometric.utils.dense_to_sparse(adjw_matrix)

In [27]:
print(edge_index)
print(edge_index.shape)
print(edge_weight)
print(edge_weight.shape)

tensor([[   0,    0,    0,  ..., 1558, 1558, 1558],
        [ 478,  536,  563,  ..., 1228, 1368, 1510]])
torch.Size([2, 34344])
tensor([0.0216, 0.0037, 0.0202,  ..., 0.0007, 0.0010, 0.0013])
torch.Size([34344])


In [15]:
index_df=pd.DataFrame(edge_index.numpy())
weight_df=pd.DataFrame(edge_weight.numpy())
adjw_df=pd.DataFrame(adjw_matrix.numpy())
index_df.to_csv("/media/alvaro-montero/9408123808121A36/Ubuntu Programs/Project_gnn/index_matrix.csv",index=False,header=False)
weight_df.to_csv("/media/alvaro-montero/9408123808121A36/Ubuntu Programs/Project_gnn/weight_matrix.csv", index=False, header=False)
adjw_df.to_csv("/media/alvaro-montero/9408123808121A36/Ubuntu Programs/Project_gnn/adjw_matrix.csv",index=False, header= False)